In [1]:
# INTRODUCCIONES GENERALES
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
#%%
def ldata(archive):
    f=open(archive)
    data=[]
    for line in f:
        line=line.strip()
        col=line.split()
        data.append(col)
    return data
#%%
prot = ldata('C:/Users/Admin/Documents/GitHub/Redes_TP2/data/yeast_AP-MS.txt')
bina = ldata('C:/Users/Admin/Documents/GitHub/Redes_TP2/data/yeast_Y2H.txt')
lit = ldata('C:/Users/Admin/Documents/GitHub/Redes_TP2/data/yeast_LIT.txt')
litreg = ldata('C:/Users/Admin/Documents/GitHub/Redes_TP2/data/yeast_LIT_Reguly.txt')
essential = ldata('C:/Users/Admin/Documents/GitHub/Redes_TP2/data/Essential_ORFs_paperHe.txt')
#%% LIMPIO LOS DATOS
ess = [a[1] for a in essential[2:1158]]
litr = [a[0:2] for a in litreg[1:]]
#%%
P = nx.Graph()
P.add_edges_from(prot)

B = nx.Graph()
B.add_edges_from(bina)

L = nx.Graph()
L.add_edges_from(lit)

LR = nx.Graph()
LR.add_edges_from(litr)

In [2]:
#les doy esencialidad como atributo
def daresencialidad(redes, ess):
    R = redes
    esencial = []#np.empty_like((len(R), 1)) ##vamos a tener que usar un diccionario porque hay distinto numero de nodos
    for i in range(len(R)):
        D = dict()
        a = list(R[i].nodes())
        for j in range(len(a)):
            D[j] = 0
            for l in range(len(ess)):
                if a[j] == ess[l]:
                    D[j] = 1
                    break
        esencial.append(D)
    
    return esencial
#%% #meto el atributo esencialidad en los nodos de cada red
def atribuir(redes, ess):
    R = redes
    for i in range(len(R)):
        for n,e,g in zip(R[i].nodes, daresencialidad(redes, ess)[i].values(), [a[1] for a in list(R[i].degree())]):
            R[i].nodes[n]['Esencialidad'] = e
            R[i].nodes[n]['Grado'] = g
            
#%%
atribuir([P,B,L,LR],ess)

Me quedo con el nombre de los nodos esenciales y no esenciales

In [4]:
R = [P,B,L,LR]
R_GC = np.empty_like(R)
for i in range(len(R)): #me quedo con la componente gigante de cada red
        R_GC[i] = max(nx.connected_component_subgraphs(R[i]), key=len)

nodos_esenciales = np.empty(4,dtype = object)
grados_esenciales = np.empty_like(nodos_esenciales)
nodos_no_esenciales = np.empty(4, dtype = object)
grados_no_esenciales = np.empty_like(nodos_no_esenciales)

for i in range(len(R_GC)):
    nodos_esenciales[i] = [nodos[0] for nodos in list(sorted(R_GC[i].nodes.data(),key = lambda x: -x[1]['Esencialidad']))][0:np.sum([c[1]['Esencialidad'] for c in list(R_GC[i].nodes.data())])]
    grados_esenciales[i] = [nodos[1]['Grado'] for nodos in list(sorted(R_GC[i].nodes.data(),key = lambda x: -x[1]['Esencialidad']))][0:np.sum([c[1]['Esencialidad'] for c in list(R_GC[i].nodes.data())])]
    nodos_no_esenciales[i] = [nodos[0] for nodos in list(sorted(R_GC[i].nodes.data(),key = lambda x: -x[1]['Esencialidad']))][np.sum([c[1]['Esencialidad'] for c in list(R_GC[i].nodes.data())]):]
    grados_no_esenciales[i] = [nodos[1]['Grado'] for nodos in list(sorted(R_GC[i].nodes.data(),key = lambda x: -x[1]['Esencialidad']))][np.sum([c[1]['Esencialidad'] for c in list(R_GC[i].nodes.data())]):]

Armo tuplas de nodo(nombre) y su grado, para que corra bien la manera de ordenarlos en orden decreciente de grado, en ambos casos

In [5]:
NG_es = [[],[],[],[]]
NG_nes = [[],[],[],[]]

def takeSecond(elem):
    return elem[1]

for i in range(len(NG_es)):
    for nodo, grado in zip(nodos_esenciales[i],grados_esenciales[i]):
        NG_es[i].append([nodo,grado])
    NG_es[i].sort(reverse=True, key= takeSecond)
    
for i in range(len(NG_nes)):
    for nnodo, ngrado in zip(nodos_no_esenciales[i],grados_no_esenciales[i]):
         NG_nes[i].append([nnodo,ngrado])
    NG_nes[i].sort(reverse = True,key=takeSecond)

Defino una función que me compare los grados de los nodos 1 a 1. Si ve que el grado esencial es mayor al no esencial en ese lugar, saca lo mejor que puede (es decir, ese valor de grado no esencial ya que están de mayor a menor). Si el grado esencial es menor al no esencial, entonces le pido que saque, con igual chance, algun nodo que tenga grado entre el no esencial (mayor en este caso) y el esencial.

In [6]:
from random import shuffle
eliminados= [max(nx.connected_component_subgraphs(P), key=len), max(nx.connected_component_subgraphs(B), key=len), max(nx.connected_component_subgraphs(L), key=len), max(nx.connected_component_subgraphs(LR), key=len)]
#en eliminados tomo las Giant components
def randomDelete(eliminados, NG_es, NG_nes):
    #veo el tamaño con el que empiezo en cada GC de la red
    size_pre_randomdelete = [eliminados[0].number_of_nodes(), eliminados[1].number_of_nodes(), eliminados[2].number_of_nodes(), eliminados[3].number_of_nodes()]
    
    for i in range(len(R)):
        #se que repito variables, perdon!
        grado_es= [elem[1] for elem in NG_es[i]] #lista de grado de los nodos esenciales para cada red i
        grado_nes = [elem[1] for elem in NG_nes[i]] #lo mismo para los no esenciales de esa misma red
        nodo_es= [elem[0] for elem in NG_es[i]] #nombre de los nodos es
        nodo_nes = [elem[0] for elem in NG_nes[i]] #nombre de los no esenciales
    
        for r in range(len(nodo_es)):
            if grado_es[r] >= grado_nes[r]:
                opciones1 = [b[0] for b in NG_nes[i] if b[1]==grado_nes[r]]  #sacame lo mejor que tengas disponible
                if len(opciones1)==1:
                    eliminados[i].remove_nodes_from(opciones1)
                else:
                    shuffle(opciones1)
                    eliminados[i].remove_nodes_from(opciones1[0])
            else: #es decir, si el grado esencial es mas chico
                opciones = [b[0] for b in NG_nes[i] if (b[1]<=grado_nes[r] and b[1]>=grado_es[r])] #dale igual oportunidad a los nodos de grado de la no esencial, hasta tan poco grado como el esencial en ese lugar
                if len(opciones)==1:
                    eliminados[i].remove_nodes_from(opciones)
                else:
                    shuffle(opciones)
                    eliminados[i].remove_nodes_from(opciones[0])
                
    size_post_randomdelete = [max(nx.connected_component_subgraphs(eliminados[0]), key=len).number_of_nodes(), max(nx.connected_component_subgraphs(eliminados[1]), key=len).number_of_nodes(), max(nx.connected_component_subgraphs(eliminados[2]), key=len).number_of_nodes(), max(nx.connected_component_subgraphs(eliminados[3]), key=len).number_of_nodes()]
    return size_pre_randomdelete, size_post_randomdelete

Lo hago muchas veces para obtener una distribucion de tamaños posibles al sacar nodos al azar


In [8]:
sizes = []
n = 0
for n in range(10):
    eliminados= [max(nx.connected_component_subgraphs(P), key=len), max(nx.connected_component_subgraphs(B), key=len), max(nx.connected_component_subgraphs(L), key=len), max(nx.connected_component_subgraphs(LR), key=len)]
    sizes.append(randomDelete(eliminados, NG_es, NG_nes))     
    n= n+1
    print(n)

1
2
3
4
5
6
7
8
9
10


In [12]:
sizes  #aca tenemos tamaño redes{p,b,l,lr} [[antes de sacarles nodos], [despues]] para cada iteracion

[([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040]),
 ([1004, 1647, 1213, 3224], [996, 1605, 1210, 3040])]

In [13]:
nodo_es= [elem[0] for elem in NG_es[0]]

In [14]:
len(nodo_es)

406

Aca no tiene sentido, porque tengo 406 nodos esenciales en la primera red, por lo que tendría que tener, cuando mínimo, 1004( nodos iniciales) - 406 (esenciales) en la red, asumiendo que los que elimino no desarman la GC.
Ahora bien, empiezo con 1004 y termino con 996 nodos, osea que eliminó 8 nodos a lo sumo (eso solo en la primera red- es decir, primer lugar de cada lista]